# 뉴스판별기


## 1. 로이터 뉴스 데이터에 대한 이해

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [93]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [94]:
fake = pd.read_csv('dataset/Fake.csv', encoding='latin1')
fake.head()


title  \
0   Donald Trump Sends Out Embarrassing New Yearâ...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obamaâs Na...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017

In [95]:
real = pd.read_csv('dataset/True.csv', encoding='latin1')
real.head()

title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017

In [96]:
real_backup = real
fake_backup = fake


In [97]:
# Null 값을 가진 샘플이 있는지 확인
real.isnull().values.any()


False

In [98]:
fake.isnull().values.any()

False

In [99]:
# 데이터 중복이 있는지 확인
fake['title'].nunique()

17903

In [100]:
real['title'].nunique()


20826

In [101]:
real['tf'] = 1
fake['tf'] = 0

In [102]:
real.drop(['text', 'subject', 'date'], axis=1, inplace=True)
fake.drop(['text', 'subject', 'date'], axis=1, inplace=True)

In [103]:
data = pd.concat([real, fake])

In [105]:
data.groupby('tf').size()


tf
0    23481
1    21417
dtype: int64

In [74]:
X_data = data['title'].values
y_data = data['tf'].values
print('뉴스 총합:', len(X_data))
print('True/False:', len(y_data))

(44898,)
(44898,)
뉴스 총합: 44898
True/False: 44898


In [75]:
X_data

array(['As U.S. budget fight looms, Republicans flip their fiscal script',
       'U.S. military to accept transgender recruits on Monday: Pentagon',
       "Senior U.S. Republican senator: 'Let Mr. Mueller do his job'",
       ...,
       'Sunnistan: US and Allied â\x80\x98Safe Zoneâ\x80\x99 Plan to Take Territorial Booty in Northern Syria',
       'How to Blow $700 Million: Al Jazeera America Finally Calls it Quits',
       '10 U.S. Navy Sailors Held by Iranian Military â\x80\x93 Signs of a Neocon Political Stunt'],
      dtype=object)

In [76]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data) # 진짜 뉴스  행을 가진 X의 각 행에 토큰화를 수행
sequences = tokenizer.texts_to_sequences(X_data) # 단어를 숫자값, 인덱스로 변환하여 저장

In [82]:
print(len(sequences))


44898


In [83]:
# 단어에 대한 인덱스
word_to_index = tokenizer.word_index
print(word_to_index) 

{'to': 1, 'trump': 2, 'in': 3, 'video': 4, 'of': 5, 'for': 6, 'on': 7, 'the': 8, 'u': 9, 's': 10, 'a': 11, 'and': 12, 'with': 13, 'says': 14, 'is': 15, 'after': 16, 'obama': 17, 'at': 18, 'as': 19, 'from': 20, 'over': 21, 'by': 22, 'house': 23, 'about': 24, 'watch': 25, 'his': 26, 'hillary': 27, 'new': 28, 'not': 29, 'will': 30, 'white': 31, 'clinton': 32, 'president': 33, 'trumpâ\x80\x99s': 34, 'be': 35, 'just': 36, 'he': 37, 'out': 38, 'it': 39, 'this': 40, 'bill': 41, 'russia': 42, 'who': 43, 'up': 44, 'that': 45, 'are': 46, 'republican': 47, 'state': 48, 'north': 49, 'against': 50, 'election': 51, 'court': 52, 'was': 53, 'her': 54, 'has': 55, 'him': 56, 'you': 57, 'senate': 58, 'breaking': 59, 'korea': 60, 'donald': 61, 'vote': 62, 'black': 63, 'news': 64, 'media': 65, 'how': 66, 'calls': 67, 'why': 68, 'anti': 69, 'tax': 70, 'no': 71, 'gop': 72, 'have': 73, 'police': 74, 'more': 75, 'republicans': 76, 'campaign': 77, 'â\x80\x93': 78, 'down': 79, 'muslim': 80, 'may': 81, 'what': 82

In [84]:
threshold = 2
total_cnt = len(word_to_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100) 

등장 빈도가 1번 이하인 희귀 단어의 수: 11233
단어 집합(vocabulary)에서 희귀 단어의 비율: 35.32056724208408
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.9594401300595174


In [85]:
vocab_size = len(word_to_index)+1
vocab_size

31804

In [106]:
X_data = sequences
max_len = max(len(l) for l in X_data)
# print('진짜 뉴스 길이 : %d' % max_len)
#
# print('진짜 뉴스 평균 : %f' % (sum(map(len, X_data))/len(X_data)))
#
# plt.hist([len(s) for s in X_data], bins=50)
# plt.xlabel('length of Data')
# plt.ylabel('number of Data')
# plt.show()

In [107]:
# 전체 데이터셋의 길이를 max_len(189)에 맞춤
data = pad_sequences(X_data, maxlen=max_len)
# data.shape

In [108]:
# 테스트용 데이터 20% 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(data, y_data, test_size=.2, random_state=2020)

In [110]:
X_train.shape, y_train.shape

((35918, 44), (35918,))

In [39]:
# X_train.reshape(18784,44)

array([[   0,    0,    0, ...,  153, 1430,    2],
       [   0,    0,    0, ...,   21, 1218, 1535],
       [   0,    0,    0, ...,   23, 5842,    2],
       ...,
       [   0,    0,    0, ...,   51,  411,  246],
       [   0,    0,    0, ...,  443, 1497,  382],
       [   0,    0,    0, ...,   23, 1437,  185]])

In [ ]:
# 학습모델 형성 

In [111]:
from keras.layers import SimpleRNN, Embedding, Dense
from keras.models import Sequential

In [118]:
model = Sequential()
model.add(Embedding(vocab_size, 44)) # 임베딩 벡터의 차원은 32
model.add(SimpleRNN(44))             # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 44)          1399376   
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 44)                3916      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 45        
Total params: 1,403,337
Trainable params: 1,403,337
Non-trainable params: 0
_________________________________________________________________


In [119]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [121]:
# 훈련용 데이터 중에서 20%는 검증 데이터로 사용
history = model.fit(X_train, y_train, epochs=10,
                    batch_size=60, validation_split=0.2, verbose=1)

C:\Users\15\.conda\envs\pyflask\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 28734 samples, validate on 7184 samples
Epoch 1/10
28734/28734 [==============================] - 10s 365us/step - loss: 0.1788 - accuracy: 0.9292 - val_loss: 0.0819 - val_accuracy: 0.9713